# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

children — количество детей в семье

days_employed — трудовой стаж в днях

dob_days — возраст клиента в годах

education — образование клиента

education_id — идентификатор образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — доход в месяц

purpose — цель получения кредита

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [2]:
import pandas as pd
data = pd.read_csv("/datasets/data.csv")
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод** 
В таблице 12 колонок - все озаглавленны ясно по смыслу. 
В некоторых данных присутствуют ошибки, например в колонке days_employed есть отрицательные значения, которые невозможны. Скорее всего какая-то нелепая ошибка, на анализ данных не влияет.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [4]:
for inc_type in data["income_type"].unique():
    data.loc[data["income_type"] == inc_type, "days_employed"] = data.loc[data["income_type"] == inc_type, "days_employed"].fillna(data.groupby("income_type")["days_employed"].median()[inc_type])    
        
print(data.isna().sum())        

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [5]:
for inc_type in data['income_type'].unique():
    data.loc[data['income_type'] == inc_type, 'total_income'] = data.loc[data['income_type'] == inc_type, 'total_income'].fillna(data.groupby('income_type')['total_income'].median()[inc_type])
                
print(data.isna().sum())        

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**
Поиск пропущенных значений показал, что в столбцах days_employed и total_income есть одинаковое количество пропусков,
заполним пропуски медианой с предварительной группировкой по типу занятости.

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
Было бы неплохо значения в столбцах days_employed и total_income привести к типу данных Int.

### Замена типа данных

In [7]:
data["days_employed"] = data["days_employed"].astype("int")
data["total_income"] = data["total_income"].astype("int")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**
Заменил вещественный тип данных на целочисленный используя метод astype().

### Обработка дубликатов

In [8]:
data.duplicated().sum()

54

In [9]:
data = data.drop_duplicates().reset_index(drop=True)

In [10]:
data.duplicated().sum()

0

In [11]:
print(data["education"].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


In [12]:
data['education'] = data['education'].str.lower()

print(data["education"].unique())    

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [13]:
print(data["purpose"].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [14]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


In [15]:
data['children'] = data['children'].replace({-1:1, 20:2})       
print(data['children'].unique())

[1 0 3 2 4 5]


**Вывод** Сначала посчитал количество дубликатов применяя к данны последовательно методы duplicated() и sum(), затем удалил их методом drop_duplicates() используя reset_index(drop=True) чтобы не создавать столбец со старыми значениями индексов. В данный момент в данных больше не встречаются дубликаты.

### Лемматизация

In [16]:
from pymystem3 import Mystem
m = Mystem()
text = " ".join(data["purpose"])
lemmas = m.lemmatize(text)

**Вывод** Произвел лемматизацию в значениях столбца с целью получения кредита, использовал библиотеку pymystem3.

In [17]:
from collections import Counter
print(Counter(lemmas))

Counter({' ': 55066, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'подержать': 853, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


**Вывод** Для подсчета лемматизированных слов использовал контейнер Counter.

### Категоризация данных

In [18]:
#произвел категоризацию самых популярных целей кредита
def lemmatize_it(text):
    lemmas = m.lemmatize(text)
    return lemmas

sorted_purposes_dict = {
    'автомобиль' : 'автомобиль', 
    'образование' : 'образование', 
    'свадьба' : 'свадьба', 
    'недвижимость' : 'недвижимость', 
    'жилье' : 'недвижимость'
}
    

def purpose_id(purpose):
    lemmatized_list = lemmatize_it(purpose)
    for lemma in lemmatized_list:
        if lemma in sorted_purposes_dict.keys():
            return sorted_purposes_dict[lemma]
        
data['purpose_id'] = data['purpose'].apply(purpose_id)       
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [19]:
pd.qcut(data.total_income, 4, labels=["низкий доход", "ниже среднего", "выше среднего", "высокий"])

0              высокий
1        ниже среднего
2        выше среднего
3              высокий
4        выше среднего
             ...      
21466          высокий
21467    выше среднего
21468     низкий доход
21469          высокий
21470     низкий доход
Name: total_income, Length: 21471, dtype: category
Categories (4, object): [низкий доход < ниже среднего < выше среднего < высокий]

In [20]:
data['total_income_id'] = pd.qcut(data.total_income, 4, labels=["низкий доход", "ниже среднего", "выше среднего", "высокий"])
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,total_income_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,ниже среднего
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,выше среднего
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,выше среднего


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [21]:
data_pivot_children = data.pivot_table(index=data["children"], columns="debt", values="income_type", aggfunc="count")
display(data_pivot_children)

debt,0,1
children,,
0,13044.0,1063.0
1,4411.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [22]:
data_pivot_children["total"] = data_pivot_children.apply(sum, axis=1)
display(data_pivot_children)

debt,0,1,total
children,,,
0,13044.0,1063.0,14107.0
1,4411.0,445.0,4856.0
2,1926.0,202.0,2128.0
3,303.0,27.0,330.0
4,37.0,4.0,41.0
5,9.0,NaN,NaN


In [23]:
data_pivot_children = data_pivot_children.fillna('0')
display(data_pivot_children)

debt,0,1,total
children,,,
0,13044.0,1063,14107
1,4411.0,445,4856
2,1926.0,202,2128
3,303.0,27,330
4,37.0,4,41
5,9.0,0,0


In [24]:
data_pivot_children.columns = ['non_debetor', 'debetor', 'total']
display(data_pivot_children)

,non_debetor,debetor,total
children,,,
0,13044.0,1063,14107
1,4411.0,445,4856
2,1926.0,202,2128
3,303.0,27,330
4,37.0,4,41
5,9.0,0,0


In [25]:
data_pivot_children['debetor'] = pd.to_numeric(data_pivot_children['debetor'])
data_pivot_children['total'] = pd.to_numeric(data_pivot_children['total'])
data_pivot_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 3 columns):
non_debetor    6 non-null float64
debetor        6 non-null float64
total          6 non-null float64
dtypes: float64(3)
memory usage: 192.0 bytes


In [26]:
data_pivot_children['debetor_share'] = data_pivot_children['debetor'] / data_pivot_children['total'] * 100
display(data_pivot_children)

,non_debetor,debetor,total,debetor_share
children,,,,
0,13044.0,1063.0,14107.0,7.535266
1,4411.0,445.0,4856.0,9.163921
2,1926.0,202.0,2128.0,9.492481
3,303.0,27.0,330.0,8.181818
4,37.0,4.0,41.0,9.756098
5,9.0,0.0,0.0,NaN


**Вывод** Результаты исследования говорят, что меньше всего процент просроченной задолженности среди клиентов, у которых нет детей, причем и общее количество всех взятых кредитов, больше всего именно в этой категории.
Среди клиетов с детьми разница в доле просроченных кредитов не так уж велика, так что можно сделать вывод, что зависимости от количества детей нет, а вот от наличия детей вообще - есть. 
Да, есть зависимость, при отсутсвии детей, доля возврата кредитов в срок - намного выше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
data_pivot_family_status = data.pivot_table(index=data["family_status"], columns="debt", values="income_type", aggfunc="count")
display(data_pivot_family_status.head(10))

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3775,388
женат / замужем,11413,931


In [28]:
data_pivot_family_status['total'] = data_pivot_family_status.apply(sum, axis=1)
display(data_pivot_family_status)

debt,0,1,total
family_status,,,
Не женат / не замужем,2536,274,2810
в разводе,1110,85,1195
вдовец / вдова,896,63,959
гражданский брак,3775,388,4163
женат / замужем,11413,931,12344


In [29]:
data_pivot_family_status.columns = ['non_debetor', 'debetor', 'total']
display(data_pivot_family_status)

,non_debetor,debetor,total
family_status,,,
Не женат / не замужем,2536,274,2810
в разводе,1110,85,1195
вдовец / вдова,896,63,959
гражданский брак,3775,388,4163
женат / замужем,11413,931,12344


In [30]:
data_pivot_family_status['debetor_share'] = data_pivot_family_status['debetor'] / data_pivot_family_status['total'] * 100
display(data_pivot_family_status)

,non_debetor,debetor,total,debetor_share
family_status,,,,
Не женат / не замужем,2536,274,2810,9.750890
в разводе,1110,85,1195,7.112971
вдовец / вдова,896,63,959,6.569343
гражданский брак,3775,388,4163,9.320202
женат / замужем,11413,931,12344,7.542126


**Вывод** Менее всего просроченной задолженности в категории вдовец/вдова, но и само количество кредитов в этой категории самое маленькое. А вот самая многочисленная категория по кредитам - женат/замужем, показывает тоже высокую долю возврата кредита в срок. Хуже всего дела с вовзратом вовремя обстоят в категориях - не женат/не змужем и гражданский брак. Можно сделать вывод, что самая надежная категория это - женат / замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
data_pivot_total_income = data.pivot_table(index=data['total_income_id'], columns="debt", values="income_type", aggfunc="count")
display(data_pivot_total_income)

debt,0,1
total_income_id,,
низкий доход,4941,427
ниже среднего,5007,483
выше среднего,4797,448
высокий,4985,383


In [32]:
data_pivot_total_income['total'] = data_pivot_total_income.apply(sum, axis=1)
display(data_pivot_total_income)

debt,0,1,total
total_income_id,,,
низкий доход,4941,427,5368
ниже среднего,5007,483,5490
выше среднего,4797,448,5245
высокий,4985,383,5368


In [33]:
data_pivot_total_income.columns = ['non_debetor', 'debetor', 'total']
display(data_pivot_total_income)

,non_debetor,debetor,total
total_income_id,,,
низкий доход,4941,427,5368
ниже среднего,5007,483,5490
выше среднего,4797,448,5245
высокий,4985,383,5368


In [34]:
data_pivot_total_income['debetor_share'] = data_pivot_total_income['debetor'] / data_pivot_total_income['total'] * 100
display(data_pivot_total_income)

,non_debetor,debetor,total,debetor_share
total_income_id,,,,
низкий доход,4941,427,5368,7.954545
ниже среднего,5007,483,5490,8.797814
выше среднего,4797,448,5245,8.541468
высокий,4985,383,5368,7.134873


**Вывод** Исследование показало, что сильной зависимости между уровнем дохода и возврата кредита в срок нет. Но все-таки лучше всего себя показывают клиенты из категории с высоким уровнем дохода, и категории - низкий доход.

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
data_pivot_purpose_id = data.pivot_table(index=data["purpose_id"], columns="debt", values="income_type", aggfunc="count")
display(data_pivot_purpose_id)

debt,0,1
purpose_id,,
автомобиль,3905,403
недвижимость,10032,782
образование,3644,370
свадьба,2149,186


In [36]:
data_pivot_purpose_id['total'] = data_pivot_purpose_id.apply(sum, axis=1)
display(data_pivot_purpose_id)

debt,0,1,total
purpose_id,,,
автомобиль,3905,403,4308
недвижимость,10032,782,10814
образование,3644,370,4014
свадьба,2149,186,2335


In [37]:
data_pivot_purpose_id.columns = ['non_debetor', 'debetor', 'total']
display(data_pivot_purpose_id)

,non_debetor,debetor,total
purpose_id,,,
автомобиль,3905,403,4308
недвижимость,10032,782,10814
образование,3644,370,4014
свадьба,2149,186,2335


In [38]:
data_pivot_purpose_id['debetor_share'] = data_pivot_purpose_id['debetor'] / data_pivot_purpose_id['total'] * 100
display(data_pivot_purpose_id)

,non_debetor,debetor,total,debetor_share
purpose_id,,,,
автомобиль,3905,403,4308,9.354689
недвижимость,10032,782,10814,7.231367
образование,3644,370,4014,9.217738
свадьба,2149,186,2335,7.965739


**Вывод** Исследование показало, что максимально вовремя возвращают кредит клиенты имеющие цель покупку недвижимости. А чаще всего просрочивают возврат кредита, клиенты бравшие его на покупку автомобиля. 
Очевидно, что зависимость от цели кредита есть.

## Шаг 4. Общий вывод

**По заданию проекта, нам предстояло выяснить - влияет ли семейное положение и количество детей клиента, на факт возврата кредита в срок.**

По результатам исследования данных о клиентах банка, можно сделать следущие общие выводы:

1) Количество детей клиента не сильно влияет на факт возврата кредита в срок, в целом в категориях с разным количеством детей (от 1 до 5), разница в доле просроченных платежей около 1%. А вот клиенты у которых вообще нет детей - намного реже просрочивают задолженность, при этом составляют основную массу клиентов.

2) Можно сказать, что семейное положение влияет на факт возврата кредита в срок, и в этом смысле самая надежная категория - "женат/замужем". Этой категории принадлежит абсолютное большинство от общего числа заемщиков, и при этом она показала почти самую высокую долю своевременного возврата задолженности, незначительно уступив только категориям: "вдовец/вдова" и "в разводе", доля которых от общего числа заемщиков значительно меньше.